# TextAttack on Keras Model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_6_Keras.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_6_Keras.ipynb)

## Training

The code below trains a basic neural network on a series of movie reviews from the IMDB dataset, loaded using Tensorflow's datasets module. Each review is encoded as a sequence of tokens corresponding to a word's index in the vocabulary. Class labels are provided, denoting a positive or negative sentiment. 

See [here](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb/load_data) for more information on the IMDB dataset. 


In [1]:
!git clone https://github.com/QData/TextAttack

Cloning into 'TextAttack'...
remote: Enumerating objects: 18529, done.
remote: Counting objects: 100% (459/459), done.
remote: Compressing objects: 100% (314/314), done.
remote: Total 18529 (delta 246), reused 260 (delta 144), pack-reused 18070
Receiving objects: 100% (18529/18529), 24.06 MiB | 14.25 MiB/s, done.
Resolving deltas: 100% (13804/13804), done.


In [2]:
cd TextAttack

/content/TextAttack


In [3]:
!git checkout api-rework

Branch 'api-rework' set up to track remote branch 'api-rework' from 'origin'.
Switched to a new branch 'api-rework'


In [4]:
!pip install ./

Processing /content/TextAttack
     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 286kB 18.4MB/s 
     |████████████████████████████████| 778kB 31.2MB/s 
     |████████████████████████████████| 225kB 26.8MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
     |████████████████████████████████| 2.3MB 29.2MB/s 
     |████████████████████████████████| 102kB 10.6MB/s 
     |████████████████████████████████| 19.7MB 97kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 798kB 27.2MB/s 
     |████████████████████████████████| 1.2MB 30.8MB/s 
     |████████████████████████████████| 983kB 30.3MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
     |████████████████████████████████| 245kB 40.5MB/s 
     |████████████████████████████████| 112kB 41.2MB/s 
     |████████████████████████████████| 3.3MB 31.2MB/s 
     |██

In [5]:
import tensorflow as tf
import keras
import numpy as np
from keras.utils import to_categorical
from textattack.models.wrappers import ModelWrapper
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019

import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout

from nltk.tokenize import word_tokenize, RegexpTokenizer


textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:16<00:00, 29.0MB/s]
textattack: Unzipping file /root/.cache/textattack/tmpipzs55s1.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


Below, we load the IMDB dataset from Tensorflow and transform it for our classifier, using a Bag-of-Words format. 

In [6]:

NUM_WORDS = 1000

(x_train_tokens, y_train), (x_test_tokens, y_test) = tf.keras.datasets.imdb.load_data(
    path="imdb.npz",
    num_words=NUM_WORDS,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)

def transform(x):
  x_transform = []
  for i, word_indices in enumerate(x):
    BoW_array = np.zeros((NUM_WORDS,))
    for index in word_indices:
      if index < len(BoW_array):
        BoW_array[index] += 1
    x_transform.append(BoW_array)
  return np.array(x_transform)
    

index = int(0.9 * len(x_train_tokens))
x_train = transform(x_train_tokens)[:index]
x_test = transform(x_test_tokens)[index:]
y_train = np.array(y_train[:index])
y_test = np.array(y_test[index:])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

vocabulary = tf.keras.datasets.imdb.get_word_index(
    path='imdb_word_index.json'
)

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

1646592/1641221 [==============================] - 0s 0us/step


With our data successfully loaded, we can now design and trained our model. 

In [7]:
#Model Created with Keras
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=NUM_WORDS))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
opt = keras.optimizers.Adam(learning_rate=0.00001)

model.compile(
 optimizer = opt,
 loss = "binary_crossentropy",
 metrics = ["accuracy"]
)


results = model.fit(
 x_train, y_train,
 epochs= 18,
 batch_size = 512,
 validation_data = (x_test, y_test)
)


print(results.history)


Epoch 1/18
44/44 [==============================] - 3s 13ms/step - loss: 0.9344 - accuracy: 0.5030 - val_loss: 0.7210 - val_accuracy: 0.5184
Epoch 2/18
44/44 [==============================] - 0s 7ms/step - loss: 0.9038 - accuracy: 0.5184 - val_loss: 0.7050 - val_accuracy: 0.5640
Epoch 3/18
44/44 [==============================] - 0s 7ms/step - loss: 0.8704 - accuracy: 0.5340 - val_loss: 0.6893 - val_accuracy: 0.5848
Epoch 4/18
44/44 [==============================] - 0s 7ms/step - loss: 0.8248 - accuracy: 0.5579 - val_loss: 0.6737 - val_accuracy: 0.6076
Epoch 5/18
44/44 [==============================] - 0s 7ms/step - loss: 0.8085 - accuracy: 0.5704 - val_loss: 0.6586 - val_accuracy: 0.6400
Epoch 6/18
44/44 [==============================] - 0s 7ms/step - loss: 0.7934 - accuracy: 0.5801 - val_loss: 0.6471 - val_accuracy: 0.6588
Epoch 7/18
44/44 [==============================] - 0s 7ms/step - loss: 0.7917 - accuracy: 0.5868 - val_loss: 0.6351 - val_accuracy: 0.6804
Epoch 8/18
44/44 [=

## Attacking

With our model trained, we can create a  `ModelWrapper` that will allow us to run TextAttack on a custom Keras model. Each `ModelWrapper` must implement a single method, `__call__`, which takes a list of strings and returns a `List`, `np.ndarray`, or `torch.Tensor` of predictions.

In [8]:
class CustomKerasModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
      
      x_transform = []
      for i, review in enumerate(text_input_list):
        tokens = [x.strip(",") for x in review.split()]
        BoW_array = np.zeros((NUM_WORDS,))
        for word in tokens:
          if word in vocabulary:
            if vocabulary[word] < len(BoW_array):
              BoW_array[vocabulary[word]] += 1            
        x_transform.append(BoW_array)
      x_transform = np.array(x_transform)
      prediction = self.model.predict(x_transform)
      return prediction


CustomKerasModelWrapper(model)(["bad bad bad bad bad", "good good good good"])

array([[0.47882336, 0.541533  ],
       [0.531164  , 0.46867093]], dtype=float32)

With our `ModelWrapper` constructed, we can use TextAttack's HuggingFaceDataset module to load reviews for testing, alongside TextAttack's PWWSRen2019 module to serve as our attack recipe. 

The attack below leverages TextAttack's `Attack` class, capable of running attacks against entire datasets. 


In [9]:
from textattack import AttackArgs
from textattack.datasets import Dataset
from textattack import Attacker

model_wrapper = CustomKerasModelWrapper(model)
dataset = HuggingFaceDataset("rotten_tomatoes", None, "test", shuffle=True)

attack = PWWSRen2019.build(model_wrapper)

attack_args = AttackArgs(num_examples=10, checkpoint_dir="checkpoints")

attacker = Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

Using custom data configuration default


textattack: Loading datasets dataset rotten_tomatoes, split test.


Dataset rotten_tomatoes_movie_review downloaded and prepared to /root/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/9c411f7ecd9f3045389de0d9ce984061a1056507703d2e3183b1ac1a90816e4d. Subsequent calls will reuse this data.


textattack: Unknown if model of class <class 'tensorflow.python.keras.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
  0%|          | 0/10 [00:00<?, ?it/s]

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



...

--------------------------------------------- Result 7 ---------------------------------------------
Positive (50%) --> Negative (51%)

throws in enough clever and unexpected twists to make the formula feel fresh .

throws in enough clever and unexpected twists to do the formula feel fresh .




[Succeeded / Failed / Skipped / Total] 5 / 0 / 4 / 9:  90%|█████████ | 9/10 [00:02<00:00,  3.12it/s]

--------------------------------------------- Result 8 ---------------------------------------------
Positive (51%) --> Negative (50%)

weighty and ponderous but every bit as filling as the treat of the title .

weighty and ponderous but every act as filling as the treat of the title .


--------------------------------------------- Result 9 ---------------------------------------------
Negative (50%) --> [SKIPPED]

a real audience-pleaser that will strike a chord with anyone who's ever waited in a doctor's office , emergency room , hospital bed or insurance company office .




[Succeeded / Failed / Skipped / Total] 6 / 0 / 4 / 10: 100%|██████████| 10/10 [00:03<00:00,  3.17it/s]

--------------------------------------------- Result 10 ---------------------------------------------
Positive (51%) --> Negative (50%)

generates an enormous feeling of empathy for its characters .

generates an enormous feel of empathy for its characters .



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 6      |
| Number of failed attacks:     | 0      |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 60.0%  |
| Accuracy under attack:        | 0.0%   |
| Attack success rate:          | 100.0% |
| Average perturbed word %:     | 9.13%  |
| Average num. words per input: | 15.4   |
| Avg num queries:              | 133.5  |
+-------------------------------+--------+


## Conclusion

Great! We trained a binary classifier, created a custom `ModelWrapper` for Keras models, and successsfully ran adversarial attacks against our trained Keras model! This serves a basic demo for how to use TextAttack within your own environments. 
